In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.auto import tqdm

from client import SESS_DIR, Client

sns.set()

In [ ]:
g = Client()

In [ ]:
res = pd.DataFrame(g.get_activities(start=0, limit=500))
mask = res["activityType"].apply(lambda d: d["typeKey"] == "running")
run = res.loc[mask]
run.head()

In [ ]:
acts = []
for i, a in tqdm(enumerate(run["activityId"])):
    cur = (
        g.get_activity(a)
        .set_index("Split")
        .drop("Summary")
        .query("Distance > 0.5")
        .reset_index()
    )
    cur["ID"] = i
    acts.append(cur)

acts = pd.concat(acts)

In [ ]:
acts["_avg_pace"] = pd.to_timedelta(acts["Avg Pace"]).apply(lambda t: t.seconds / 60)

In [ ]:
acts.groupby("Split")[["_avg_pace", "Avg HR", "Max HR"]].agg(
    ["count", "min", "max", "mean", "std", "median"]
)